In [1]:
# !pip install accelerate -U -q
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import math
import json
import torch
import pickle
import pandas as pd
import torch.nn as nn
import torch.nn.init as init
# from transformers import AutoModelForCausalLM, GemmaConfig, AutoModel, MistralConfig, MistralModel, MistralForCausalLM
from transformers import AutoTokenizer, LlamaConfig, LlamaForCausalLM

/home/preyum.kumar/CollegeWork/computer vision/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Code 1

In [3]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [4]:
tokenizer = AutoTokenizer.from_pretrained("./Tokenizer3")

In [5]:
tokenizer.pad_token_id = tokenizer.eos_token_id
len(tokenizer.vocab)

128001

In [6]:
config = LlamaConfig(hidden_size=256,
                     vocab_size=128000,
                     num_attention_heads=8,
                     num_key_value_heads=8,
                     num_hidden_layers=16,
                     intermediate_size=512,
                     max_position_embeddings=2048)
config

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 32,
  "hidden_act": "silu",
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 8,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.46.2",
  "use_cache": true,
  "vocab_size": 128000
}

In [7]:
model_mis = LlamaForCausalLM(config)

In [8]:
for i,j in model_mis.named_parameters():
  if j.requires_grad and len(j.size()) > 1:
    init.xavier_uniform_(j.data)

In [9]:
total_param=0
for i,j in model_mis.named_parameters():
    total_param += j.numel()
print(total_param/(10**6))

76.030208


In [ ]:
model_mis.save_pretrained("./marathi_model_large_5epoch")
tokenizer.save_pretrained("./marathi_model_large_5epoch")

('./marathi_model_large_1epoch/tokenizer_config.json',
 './marathi_model_large_1epoch/special_tokens_map.json',
 './marathi_model_large_1epoch/tokenizer.json')

### Code 2

In [11]:
# !pip install datasets -q

In [12]:
# import pandas as pd
# from datasets import load_dataset
# from datasets import Dataset, DatasetDict

In [13]:
# df = pd.read_csv("extracted_data.csv")

In [14]:
# df.head()

In [15]:
# df = df.iloc[:2000, :]
# df

In [16]:
# df["text"] = df["text"].str.strip() + "<end_of_sen>"

In [17]:
# tokenizer = AutoTokenizer.from_pretrained("Tokenizer3")

In [18]:
# df.iloc[0, -1]

In [19]:
# input_ids = tokenizer(df["text"].to_list()[:])["input_ids"]

In [20]:
# token_list = []
# for i in input_ids:
#   token_list.extend(i)

In [21]:
# token_list = token_list + [3] * 180
# len(token_list)

In [22]:
# df = pd.DataFrame(columns=["input_ids"])
# df.shape

In [23]:
# # import numpy as np
# # from tqdm import tqdm

# # context_len = 256      ## Taking less because I have less data
# # token_batch = []
# # with tqdm(total=len(input_ids)) as pbar:
# #   for i in input_ids: 
# #     token_batch.append(token_list[:context_len])
# #     token_list = token_list[context_len:]
# #     pbar.update(1)
# # token_list.extend([3, 3])
# # token_list = np.array(token_list)
# # token_batch = np.array_split(token_list, len(token_list) // context_len + 1)
# import numpy as np
# from tqdm import tqdm

# context_len = 256  # Taking less because I have less data
# token_batch = []

# # Calculate the number of batches
# num_batches = (len(token_list) + context_len - 1) // context_len

# with tqdm(total=num_batches) as pbar:
#     for i in range(num_batches):
#         token_batch.append(token_list[i * context_len:(i + 1) * context_len])
#         pbar.update(1)

# # Convert to numpy array if needed
# # token_batch = np.array(token_batch)

In [24]:
# len(token_batch), len(token_batch[-1])

In [25]:
# df["input_ids"] = token_batch
# df

In [26]:
# attn_mask = [[1]*context_len]*len(df)

In [27]:
# df["attention_mask"] = attn_mask
# df['labels'] = df['input_ids']

In [28]:
# df.head()

In [29]:
# hf_dataset = Dataset.from_pandas(df)
# hf_dataset

In [30]:
# split_dataset = hf_dataset.train_test_split(test_size=0.1)  # Adjust test_size as needed

# train_dataset = split_dataset['train']
# eval_dataset = split_dataset['test']

In [31]:
# train_dataset

In [32]:
# train_dataset.to_parquet("marathi_dataset_token_train_large.parquet")
# eval_dataset.to_parquet("marathi_dataset_token_test_large.parquet")

### Code 3

In [33]:
from transformers import Trainer, TrainingArguments

In [34]:
#!pip install -q bitsandbytes trl peft -U

In [35]:
from datasets import load_dataset
dataset = load_dataset('parquet', data_files=
 {'train': './marathi_dataset_token_train_large.parquet',
  'test': './marathi_dataset_token_test_large.parquet'})

In [36]:
import wandb
# 6d0aee422e6256561f2b4ef7fd7ddb3df99f44fb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mukul-potta (mukul-potta-indian-institute-of-technology-gandhinagar). Use `wandb login --relogin` to force relogin


True

In [37]:
#!pip install -q bitsandbytes trl peft -U
import os
# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="marathi_nlp"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="checkpoint"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="all"

In [ ]:
training_args = TrainingArguments(
    output_dir="./marathi_model_large_5epoch",
    overwrite_output_dir=True,
    num_train_epochs=5,
    eval_strategy="steps",
    save_strategy='epoch',
    logging_steps=2393,
    # logging_steps=10,
    eval_steps=2393,
    # eval_steps=10,
    learning_rate=2e-3,
    bf16=True,
    do_train=True,
    do_eval=True,
    # per_device_train_batch_size=32,
    per_device_train_batch_size=16,
    save_steps=5000,
    save_total_limit=2,
    report_to="wandb",
    run_name='final_MCB_5epoch',
    # report_to="none",
)

In [ ]:
per_log = []
perplex = []
eval_loss_list = []

In [ ]:
from transformers.trainer_callback import TrainerControl, TrainerState
from transformers.training_args import TrainingArguments


class PerplexityCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, model=None, **kwargs):
        if state.log_history:  
            for entry in state.log_history:
                if "eval_loss" in entry:
                    print(entry)
                    eval_loss = entry["eval_loss"]
                    eval_loss_list.append(eval_loss)
                    perplexity = math.exp(eval_loss)
                    step = entry["step"]
                    per_log.append(step)
                    perplex.append(perplexity)
                    print(f"Step {step}")
                    print(f"Perplexity = {perplexity:.4f}")

In [ ]:
train_data = dataset['train']
train_data

In [ ]:
eval_data = dataset['test']
eval_data

In [ ]:
trainer = Trainer(
    model=model_mis,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    processing_class=tokenizer,
    callbacks=[PerplexityCallback()],
)

/tmp/ipykernel_1826121/432772971.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [40]:
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=no,
eval_use_gather_object=Fals

In [41]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
trainer.train()

Step,Training Loss
100,9.623600
200,9.186700
300,8.964500
400,8.773300
500,8.586800
600,8.390800
700,8.238900
800,8.116500
900,8.005200
1000,7.891600


wandb: Adding directory to artifact (./marathi_model_large_1epoch/checkpoint-4000)... Done. 11.7s
wandb: Adding directory to artifact (./marathi_model_large_1epoch/checkpoint-8000)... Done. 12.0s


In [ ]:
# custom_input = "The woman who died after falling from a sofa"
# input_dict = {'text': [custom_input]}

In [ ]:
# input_dict = {'input_ids': [tokenizer.encode(custom_input)]}
# input_dict

In [ ]:
# custom_dataset = Dataset.from_dict(input_dict)

In [ ]:
# predictions = trainer.predict(custom_dataset)

In [ ]:
# generated_outputs = predictions.predictions  # This will be logits

In [ ]:
# output_ids = torch.argmax(torch.tensor(generated_outputs), dim=2)

In [ ]:
# tokenizer.decode(output_ids[0])

In [ ]:
# output_ids

In [ ]:
trainer.save_model("./trained_model_large_5epoch")
tokenizer.save_pretrained("./trained_model_large_5epoch")